In [1]:
!pip install torch==2.0.1 torchvision==0.15.2 einops av # Adjust if needed
!apt-get install ffmpeg  # for video reading if not installed

import os
import glob
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader, random_split
from einops import rearrange
import math
from torch.optim.lr_scheduler import LambdaLR

from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#=============================
# Configurations
#=============================
DATA_DIR = '/content/drive/MyDrive/PhD/VideoDataset'  # Adjust to your data path
NUM_FRAMES = 16      # Number of frames to sample from each video
IMAGE_SIZE = 224      # Spatial size of each frame
BATCH_SIZE = 2        # Adjust depending on your GPU memory
EPOCHS = 20
LR = 1e-4
NUM_WORKERS = 2
SEED = 42

random.seed(SEED)
torch.manual_seed(SEED)

In [3]:
#=============================
# Video Dataset
#=============================
class VideoDataset(Dataset):
    def __init__(self, root_dir, num_frames=16, image_size=224, transform=None):
        self.root_dir = root_dir
        self.num_frames = num_frames
        self.image_size = image_size
        self.transform = transform

        # Debug: Start dataset initialization
        print(f"Initializing VideoDataset with root_dir: {self.root_dir}")

        # Collect all video paths with labels
        self.samples = []
        classes = ['Fall', 'ADL']
        for label_idx, cls in enumerate(classes):
            class_dir = os.path.join(root_dir, cls)
            if not os.path.exists(class_dir):
                #print(f"Warning: Class directory {class_dir} does not exist!")
                continue

            #print(f"Scanning class '{cls}' in {class_dir}...")
            for sbj_dir in glob.glob(os.path.join(class_dir, '*')):
                if not os.path.isdir(sbj_dir):
                    #print(f"Skipping non-directory {sbj_dir}")
                    continue

                #print(f"  Found subject directory: {sbj_dir}")
                for act_dir in glob.glob(os.path.join(sbj_dir, 'ACT*')):
                    if not os.path.isdir(act_dir):
                        #print(f"  Skipping non-directory {act_dir}")
                        continue

                    #print(f"    Found activity folder: {act_dir}")
                    for video_path in glob.glob(os.path.join(act_dir, '*.mp4')):
                        #print(f"      Found video file: {video_path}")
                        self.samples.append((video_path, label_idx))

        # Debug: Print the number of samples found
        print(f"Total samples found: {len(self.samples)}")
        if len(self.samples) == 0:
            print("Error: No video files found! Please check your dataset structure.")

        # Shuffle samples
        random.shuffle(self.samples)

    def __len__(self):
        return len(self.samples)

    def _uniform_sample_indices(self, total_frames):
        # Uniformly sample self.num_frames from total_frames
        if total_frames < self.num_frames:
            # If video shorter, we just sample frames repeatedly
            indices = list(range(total_frames))
            while len(indices) < self.num_frames:
                indices.append(indices[-1])  # or wrap around
            return indices
        else:
            step = total_frames / self.num_frames
            indices = [int(i*step) for i in range(self.num_frames)]
            return indices

    def __getitem__(self, idx):
        video_path, label = self.samples[idx]
        # Read video with torchvision.io
        # video is returned as (T, H, W, C), audio is ignored
        vid, _, info = torchvision.io.read_video(video_path, pts_unit='sec')
        # vid shape: (num_total_frames, H, W, C)

        total_frames = vid.shape[0]
        frame_indices = self._uniform_sample_indices(total_frames)
        vid = vid[frame_indices]  # (num_frames, H, W, C)

        # Convert to float and [0,1]
        vid = vid.float() / 255.0

        # Apply spatial transform (resize, center crop, etc.)
        # We'll do a simple resize and center crop with transforms.
        # We'll treat each frame as an image and apply the same transform.
        if self.transform:
            # apply transform frame by frame
            # The transform is expected to handle a single image,
            # but we can apply it to each frame in a loop.
            transformed_frames = []
            for f in vid:
                f = self.transform(f.permute(2,0,1))  # C,H,W
                transformed_frames.append(f)
            vid = torch.stack(transformed_frames, dim=0)  # (T, C, H, W)
        else:
            # default transform
            transform = T.Compose([
                T.Resize((self.image_size, self.image_size)),
            ])
            transformed_frames = []
            for f in vid:
                f = f.permute(2,0,1)
                f = transform(f)
                transformed_frames.append(f)
            vid = torch.stack(transformed_frames, dim=0)  # (T, C, H, W)

        # Output shape: (T, C, H, W), label: int
        return vid, label

In [4]:
#=============================
# Transforms
#=============================
train_transform = T.Compose([
    T.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)),
    T.RandomHorizontalFlip(p=0.5),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
])

val_transform = T.Compose([
    T.Resize((IMAGE_SIZE, IMAGE_SIZE)),
])

In [5]:
def warmup_cosine_lr_lambda(epoch, total_epochs, warmup_epochs):
    """
    Args:
        epoch (int): Current epoch (0-based).
        total_epochs (int): Total number of epochs.
        warmup_epochs (int): Number of epochs for linear warm-up.

    Returns:
        float: A multiplier for the base learning rate.
    """
    # 1) Warm-up phase
    if epoch < warmup_epochs:
        return float(epoch) / float(max(1, warmup_epochs))

    # 2) Cosine decay phase
    # "progress" goes from 0 (end of warm-up) to 1 (end of training)
    progress = float(epoch - warmup_epochs) / float(max(1, total_epochs - warmup_epochs))

    # Cosine decay from 1.0 (start) down to ~0 (end)
    return 0.5 * (1.0 + math.cos(math.pi * progress))

In [6]:
#=============================
# Create Dataset & Split
#=============================
dataset = VideoDataset(DATA_DIR, num_frames=NUM_FRAMES, image_size=IMAGE_SIZE, transform=train_transform)
val_ratio = 0.2
val_size = int(len(dataset)*val_ratio)
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=torch.Generator().manual_seed(SEED))

# For validation dataset, use val_transform (if different)
val_dataset.dataset.transform = val_transform

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

from torch.cuda.amp import autocast, GradScaler

scaler = GradScaler()

Initializing VideoDataset with root_dir: /content/drive/MyDrive/PhD/VideoDataset
Total samples found: 2014


In [7]:
#=============================
# Vision Transformer Model (Simplified)
#=============================
import torch.nn as nn

class PatchEmbedding(nn.Module):
    def __init__(self, in_channels=3, patch_size=16, emb_dim=768, frames=16, image_size=224):
        super().__init__()
        self.patch_size = patch_size
        self.frames = frames
        self.num_patches = (image_size // patch_size) ** 2
        self.linear = nn.Linear(in_channels * patch_size * patch_size, emb_dim)

        # Positional embeddings: We'll have a separate temporal dimension or flatten T with patches
        # Flatten everything into a sequence: total length = T * (H*W / patch_size^2)
        self.pos_embed = nn.Parameter(torch.randn(1, frames*self.num_patches, emb_dim))

    def forward(self, x):
        # x: (B, T, C, H, W)
        B, T, C, H, W = x.shape
        # split into patches
        # rearrange with einops: (B, T, C, H, W) -> (B, T*(H/ps)*(W/ps), C*ps*ps)
        p = self.patch_size
        x = rearrange(x, 'b t c (h p1) (w p2) -> b (t h w) (c p1 p2)', p1=p, p2=p)
        x = self.linear(x)  # (B, T*num_patches, emb_dim)

        # Add positional embeddings
        x = x + self.pos_embed[:, :x.size(1), :]  # Just truncate if mismatch
        return x

class TransformerEncoderBlock(nn.Module):
    def __init__(self, emb_dim=768, num_heads=8, mlp_ratio=4.0, dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(emb_dim)
        self.attn = nn.MultiheadAttention(emb_dim, num_heads, dropout=dropout, batch_first=True)
        self.norm2 = nn.LayerNorm(emb_dim)

        self.mlp = nn.Sequential(
            nn.Linear(emb_dim, int(emb_dim*mlp_ratio)),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(int(emb_dim*mlp_ratio), emb_dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        # x: (B, N, C)
        x = x + self.attn(self.norm1(x), self.norm1(x), self.norm1(x))[0]
        x = x + self.mlp(self.norm2(x))
        return x

class VisionTransformer(nn.Module):
    def __init__(self, num_classes=2, emb_dim=768, depth=6, num_heads=8, mlp_ratio=4.0, frames=16, image_size=224, patch_size=16):
        super().__init__()
        self.patch_embed = PatchEmbedding(
            in_channels=3, patch_size=patch_size, emb_dim=emb_dim, frames=frames, image_size=image_size
        )
        self.cls_token = nn.Parameter(torch.randn(1, 1, emb_dim))
        self.blocks = nn.ModuleList([
            TransformerEncoderBlock(emb_dim=emb_dim, num_heads=num_heads, mlp_ratio=mlp_ratio) for _ in range(depth)
        ])
        self.norm = nn.LayerNorm(emb_dim)
        self.fc = nn.Linear(emb_dim, num_classes)

    def forward(self, x):
        # x: (B, T, C, H, W)
        B = x.size(0)
        x = self.patch_embed(x)  # (B, N, D) where N = T * (H/ps)*(W/ps)
        # prepend cls token
        cls_tokens = self.cls_token.expand(B, -1, -1)  # (B, 1, D)
        x = torch.cat([cls_tokens, x], dim=1)  # (B, 1+N, D)

        for blk in self.blocks:
            x = blk(x)

        x = self.norm(x)
        # cls token is at x[:,0]
        cls_out = x[:, 0]
        out = self.fc(cls_out)
        return out

In [8]:
# Experiments: https://chatgpt.com/c/677d688e-7144-8006-8335-b41a18edb22c
#=============================
# Training and Evaluation
#=============================
!pip install av

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VisionTransformer(
    num_classes=2,
    emb_dim=512,
    depth=8,
    num_heads=4,
    mlp_ratio=3.0,
    frames=NUM_FRAMES,
    image_size=IMAGE_SIZE,
    patch_size=16
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

warmup_epochs = 3  # Number of epochs for warm-up, tweak as needed

scheduler = LambdaLR(
    optimizer,
    lr_lambda=lambda epoch: warmup_cosine_lr_lambda(
        epoch=epoch,
        total_epochs=EPOCHS,
        warmup_epochs=warmup_epochs
    )
)

def train_one_epoch(model, loader, optimizer, criterion, device, scaler):
    model.train()
    total_loss, total_correct = 0.0, 0

    for batch_idx, (vids, labels) in enumerate(loader):
        vids, labels = vids.to(device), labels.to(device)

        # Reset gradients
        optimizer.zero_grad()

        # Forward pass (mixed precision)
        with autocast():
            preds = model(vids)
            loss = criterion(preds, labels)

        # Backward pass with scaled loss
        scaler.scale(loss).backward()

        # Step optimizer
        scaler.step(optimizer)
        # Update the scale for next iteration
        scaler.update()

        # Accumulate stats
        total_loss += loss.item() * vids.size(0)
        total_correct += (preds.argmax(dim=1) == labels).sum().item()

        # Optional per-batch logging
        print(f"[Train] Batch {batch_idx+1}/{len(loader)} - "
              f"Loss: {loss.item():.4f}, "
              f"Acc: {((preds.argmax(dim=1) == labels).sum().item()/vids.size(0)):.4f}")

    avg_loss = total_loss / len(loader.dataset)
    avg_acc = total_correct / len(loader.dataset)
    return avg_loss, avg_acc

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, total_correct = 0.0, 0

    with torch.no_grad():
        for batch_idx, (vids, labels) in enumerate(loader):
            vids, labels = vids.to(device), labels.to(device)

            # Forward pass (mixed precision)
            with autocast():
                preds = model(vids)
                loss = criterion(preds, labels)

            total_loss += loss.item() * vids.size(0)
            total_correct += (preds.argmax(dim=1) == labels).sum().item()

            print(f"[Eval] Batch {batch_idx+1}/{len(loader)} - "
                  f"Loss: {loss.item():.4f}, "
                  f"Acc: {((preds.argmax(dim=1) == labels).sum().item()/vids.size(0)):.4f}")

    avg_loss = total_loss / len(loader.dataset)
    avg_acc = total_correct / len(loader.dataset)
    return avg_loss, avg_acc

for epoch in range(EPOCHS):
    print(f"Starting Epoch {epoch+1}/{EPOCHS}")

    # ----------------------
    # Training
    # ----------------------
    train_loss, train_acc = train_one_epoch(
        model, train_loader, optimizer, criterion, device, scaler
    )

    # ----------------------
    # Validation
    # ----------------------
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)

    # ----------------------
    # Step the Scheduler
    # ----------------------
    scheduler.step()

    # Print your metrics
    print(f"Epoch {epoch+1}/{EPOCHS}:")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"  Val   Loss: {val_loss:.4f}, Val   Acc: {val_acc:.4f}")
    current_lr = scheduler.get_last_lr()[0]  # For Adam there's typically one group
    print(f"  Current Learning Rate: {current_lr:.6f}")

# After the training loop finishes:

# 1. Optionally save the trained model weights
MODEL_PATH = "/content/drive/MyDrive/model_weights.pth"
torch.save(model.state_dict(), MODEL_PATH)

print(f"Model weights saved to {MODEL_PATH}")

# 2. Evaluate on a test set, if you have one
# Make sure you've created `val_loader` similar to `train_loader` and `val_loader`.
try:
    test_loss, test_acc = evaluate(model, val_loader, criterion, device)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")
except NameError:
    print("No test_loader found. Skipping test evaluation.")

Starting Epoch 1/20


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

[Train] Batch 1/806 - Loss: 1.4062, Acc: 0.0000
[Train] Batch 2/806 - Loss: 0.3123, Acc: 1.0000
[Train] Batch 3/806 - Loss: 0.7290, Acc: 0.5000
[Train] Batch 4/806 - Loss: 0.8326, Acc: 0.5000
[Train] Batch 5/806 - Loss: 1.2603, Acc: 0.0000
[Train] Batch 6/806 - Loss: 1.4141, Acc: 0.0000
[Train] Batch 7/806 - Loss: 0.3140, Acc: 1.0000
[Train] Batch 8/806 - Loss: 0.6844, Acc: 0.5000
[Train] Batch 9/806 - Loss: 0.8395, Acc: 0.5000
[Train] Batch 10/806 - Loss: 0.7378, Acc: 0.5000
[Train] Batch 11/806 - Loss: 1.4043, Acc: 0.0000
[Train] Batch 12/806 - Loss: 0.9146, Acc: 0.5000
[Train] Batch 13/806 - Loss: 0.3218, Acc: 1.0000
[Train] Batch 14/806 - Loss: 0.2795, Acc: 1.0000
[Train] Batch 15/806 - Loss: 1.3364, Acc: 0.0000
[Train] Batch 16/806 - Loss: 1.3022, Acc: 0.0000
[Train] Batch 17/806 - Loss: 0.7460, Acc: 0.5000
[Train] Batch 18/806 - Loss: 0.7653, Acc: 0.5000
[Train] Batch 19/806 - Loss: 1.3013, Acc: 0.0000
[Train] Batch 20/806 - Loss: 0.7771, Acc: 0.5000
[Train] Batch 21/806 - Loss: 

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

[Eval] Batch 1/201 - Loss: 0.8330, Acc: 0.5000
[Eval] Batch 2/201 - Loss: 0.8309, Acc: 0.5000
[Eval] Batch 3/201 - Loss: 0.8239, Acc: 0.5000
[Eval] Batch 4/201 - Loss: 0.3221, Acc: 1.0000
[Eval] Batch 5/201 - Loss: 0.8180, Acc: 0.5000
[Eval] Batch 6/201 - Loss: 0.8174, Acc: 0.5000
[Eval] Batch 7/201 - Loss: 0.7898, Acc: 0.5000
[Eval] Batch 8/201 - Loss: 1.3013, Acc: 0.0000
[Eval] Batch 9/201 - Loss: 0.8082, Acc: 0.5000
[Eval] Batch 10/201 - Loss: 1.3076, Acc: 0.0000
[Eval] Batch 11/201 - Loss: 0.8011, Acc: 0.5000
[Eval] Batch 12/201 - Loss: 1.2656, Acc: 0.0000
[Eval] Batch 13/201 - Loss: 0.3307, Acc: 1.0000
[Eval] Batch 14/201 - Loss: 0.8147, Acc: 0.5000
[Eval] Batch 15/201 - Loss: 0.7749, Acc: 0.5000
[Eval] Batch 16/201 - Loss: 0.8009, Acc: 0.5000
[Eval] Batch 17/201 - Loss: 0.8064, Acc: 0.5000
[Eval] Batch 18/201 - Loss: 0.8119, Acc: 0.5000
[Eval] Batch 19/201 - Loss: 0.8258, Acc: 0.5000
[Eval] Batch 20/201 - Loss: 0.3202, Acc: 1.0000
[Eval] Batch 21/201 - Loss: 1.3105, Acc: 0.0000
[

...

[Train] Batch 1/806 - Loss: 0.0000, Acc: 1.0000
[Train] Batch 2/806 - Loss: 0.0000, Acc: 1.0000
[Train] Batch 3/806 - Loss: 0.0000, Acc: 1.0000
[Train] Batch 4/806 - Loss: 0.0000, Acc: 1.0000
[Train] Batch 5/806 - Loss: 0.0000, Acc: 1.0000
[Train] Batch 6/806 - Loss: 0.0000, Acc: 1.0000
[Train] Batch 7/806 - Loss: 0.0000, Acc: 1.0000
[Train] Batch 8/806 - Loss: 0.0001, Acc: 1.0000
[Train] Batch 9/806 - Loss: 0.0003, Acc: 1.0000
[Train] Batch 10/806 - Loss: 0.0000, Acc: 1.0000
[Train] Batch 11/806 - Loss: 0.0000, Acc: 1.0000
[Train] Batch 12/806 - Loss: 0.0001, Acc: 1.0000
[Train] Batch 13/806 - Loss: 0.0000, Acc: 1.0000
[Train] Batch 14/806 - Loss: 0.0000, Acc: 1.0000
[Train] Batch 15/806 - Loss: 0.0007, Acc: 1.0000
[Train] Batch 16/806 - Loss: 0.0000, Acc: 1.0000
[Train] Batch 17/806 - Loss: 0.0001, Acc: 1.0000
[Train] Batch 18/806 - Loss: 0.0000, Acc: 1.0000
[Train] Batch 19/806 - Loss: 0.0000, Acc: 1.0000
[Train] Batch 20/806 - Loss: 0.0000, Acc: 1.0000
[Train] Batch 21/806 - Loss: 

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

[Eval] Batch 1/201 - Loss: 0.0011, Acc: 1.0000
[Eval] Batch 2/201 - Loss: 0.0004, Acc: 1.0000
[Eval] Batch 3/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 4/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 5/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 6/201 - Loss: 0.0005, Acc: 1.0000
[Eval] Batch 7/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 8/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 9/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 10/201 - Loss: 0.0005, Acc: 1.0000
[Eval] Batch 11/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 12/201 - Loss: 0.0002, Acc: 1.0000
[Eval] Batch 13/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 14/201 - Loss: 0.0001, Acc: 1.0000
[Eval] Batch 15/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 16/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 17/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 18/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 19/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 20/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 21/201 - Loss: 0.0000, Acc: 1.0000
[

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

[Eval] Batch 1/201 - Loss: 0.0011, Acc: 1.0000
[Eval] Batch 2/201 - Loss: 0.0004, Acc: 1.0000
[Eval] Batch 3/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 4/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 5/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 6/201 - Loss: 0.0005, Acc: 1.0000
[Eval] Batch 7/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 8/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 9/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 10/201 - Loss: 0.0005, Acc: 1.0000
[Eval] Batch 11/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 12/201 - Loss: 0.0002, Acc: 1.0000
[Eval] Batch 13/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 14/201 - Loss: 0.0001, Acc: 1.0000
[Eval] Batch 15/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 16/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 17/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 18/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 19/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 20/201 - Loss: 0.0000, Acc: 1.0000
[Eval] Batch 21/201 - Loss: 0.0000, Acc: 1.0000
[

In [9]:
# 3. Inference on a new video:
# Suppose you have a new video file:
# e.g., "/content/drive/MyDrive/new_video.mp4"
NEW_VIDEO_PATH = "/content/drive/MyDrive/PhD/Dataset/Fall/SBJ_08_LOC3/ACT11_B_1/20240914132224.mp4"  # Adjust path accordingly

model.eval()
with torch.no_grad():
    # Read and preprocess the single new video similarly to how your dataset does it
    vid, _, info = torchvision.io.read_video(NEW_VIDEO_PATH, pts_unit='sec')
    vid = vid.float() / 255.0

    # Sample frames uniformly
    total_frames = vid.shape[0]
    # We'll reuse the same sampling function logic used in the dataset
    def uniform_sample_indices(total_frames, num_frames=16):
        if total_frames < num_frames:
            indices = list(range(total_frames))
            while len(indices) < num_frames:
                indices.append(indices[-1])
            return indices
        else:
            step = total_frames / num_frames
            indices = [int(i*step) for i in range(num_frames)]
            return indices

    frame_indices = uniform_sample_indices(total_frames, num_frames=16)
    vid = vid[frame_indices]  # (T, H, W, C)

    # Transform the video frames: resize to IMAGE_SIZE and convert shape
    transform = T.Resize((IMAGE_SIZE, IMAGE_SIZE))
    frames_transformed = []
    for f in vid:
        f = f.permute(2, 0, 1)  # C,H,W
        f = transform(f)
        frames_transformed.append(f)
    vid = torch.stack(frames_transformed, dim=0)  # (T, C, H, W)

    vid = vid.unsqueeze(0).to(device)  # Add batch dimension
    prediction = model(vid)
    pred_class = prediction.argmax(dim=1).item()
    # Assuming Fall = 0, ADL = 1 (or vice versa depending on how you labeled)
    label_map = {0: "Fall", 1: "ADL"}
    print("Predicted Class for new video:", label_map[pred_class])

Predicted Class for new video: Fall


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
